In [5]:
import numpy as np

# Step 1: Compute A^T * A
def compute_ata(matrix):
    return np.dot(matrix.T, matrix)

# Step 2: Compute eigenvalues and eigenvectors of A^T * A
def calculate_eigen(matrix):
    eigenvalues, eigenvectors = np.linalg.eigh(matrix)
    return eigenvalues, eigenvectors

# Step 3: Sort eigenvalues and corresponding eigenvectors in descending order
def sort_eigen(eigenvalues, eigenvectors):
    sorted_indices = np.argsort(eigenvalues)[::-1]
    sorted_eigenvalues = eigenvalues[sorted_indices]
    sorted_eigenvectors = eigenvectors[:, sorted_indices]
    return sorted_eigenvalues, sorted_eigenvectors

# Step 6: Compute U using the formula u_i = (1/σ_i) * A * v_i
def compute_u(matrix, singular_values, V):
    U = []
    for i, sigma in enumerate(singular_values):
        if sigma > 1e-10:  # Avoid division by zero for very small singular values
            u_i = (1 / sigma) * np.dot(matrix, V[:, i])
            U.append(u_i)
    U = np.array(U).T  # Make U a matrix
    return U

# Step 4 & 5: Compute singular values (square root of eigenvalues) and V
def compute_svd(matrix):
    # Compute A^T * A
    ATA = compute_ata(matrix)
    
    # Find eigenvalues and eigenvectors
    eigenvalues, eigenvectors = calculate_eigen(ATA)
    
    # Sort eigenvalues and eigenvectors in descending order
    sorted_eigenvalues, V = sort_eigen(eigenvalues, eigenvectors)
    
    # Compute singular values as square roots of eigenvalues
    singular_values = np.sqrt(np.maximum(sorted_eigenvalues, 0))  # Avoid numerical issues
    
    # Compute U using the formula
    U = compute_u(matrix, singular_values, V)
    
    # Form Σ as diagonal matrix of singular values
    Sigma = np.zeros_like(matrix, dtype=float)
    min_dim = min(matrix.shape)
    np.fill_diagonal(Sigma, singular_values[:min_dim])
    
    return U, Sigma, V.T

# Verification of A = U Σ V^T
def verify_svd(matrix, U, Sigma, V_T):
    reconstructed_matrix = U @ Sigma @ V_T
    return matrix==reconstructed_matrix

# Main Program
if __name__ == "__main__":
    # Input matrix A
    A = np.array([[1, 2], [3, 4]])
    print("Matrix A:")
    print(A)
    
    # Compute SVD
    U, Sigma, V_T = compute_svd(A)
    
    # Output results
    print("\nU (Left Singular Vectors):")
    print(U)
    print("\nΣ (Singular Values Diagonal Matrix):")
    print(Sigma)
    print("\nV^T (Right Singular Vectors Transposed):")
    print(V_T)
    
    # Verify SVD
    is_verified = verify_svd(A, U, Sigma, V_T)
    print(f"\nVerification of A = U Σ V^T: {'Result Ok' if is_verified else 'Mismatch'}")


Matrix A:
[[1 2]
 [3 4]]

U (Left Singular Vectors):
[[ 0.40455358  0.9145143 ]
 [ 0.9145143  -0.40455358]]

Σ (Singular Values Diagonal Matrix):
[[5.4649857  0.        ]
 [0.         0.36596619]]

V^T (Right Singular Vectors Transposed):
[[ 0.57604844  0.81741556]
 [-0.81741556  0.57604844]]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
import numpy as np

def row_column_input():
    check = True
    while check:
        try:
            num_of_row = int(input("Enter number of rows: "))
            num_of_col = int(input("Enter number of columns: "))
            check = False
        except ValueError as e:
            print(f"Error: {e}.")
    return num_of_row, num_of_col

def input_matrix(num_rows, num_cols):
    matrix = []
    i = 0
    while i < num_rows:
        try:
            row = list(map(int, input(f"Row {i+1}, enter values separated by space ({num_cols}): ").split()))
            if num_cols != len(row):
                raise ValueError(f"Expected {num_cols} values but got {len(row)} values.")
            matrix.append(row)
            i += 1  
        except ValueError as e:
            print(f"Error: {e}. Please re-enter Row {i+1}.")
    return np.array(matrix)

def is_svd_possible(matrix):
    """
    Check if SVD is possible: non-empty, numerical, and 2D.
    """
    if matrix.size == 0:
        print("Matrix is empty!")
        return False
    if len(matrix.shape) != 2:
        print("Matrix is not 2D!")
        return False
    return True

def calculate_eigen(matrix):
    """
    Calculate eigenvalues and eigenvectors of a given matrix.
    """
    eigvals, eigvecs = np.linalg.eig(matrix)
    return eigvals, eigvecs

def svd_decomposition(matrix):
    """
    Perform SVD decomposition by manually calculating U, Σ, and V^T.
    """
    ata = np.dot(matrix.T, matrix)  # A^T * A
    aat = np.dot(matrix, matrix.T)  # A * A^T

    eigvals_v, eigvecs_v = calculate_eigen(ata)  # Eigen decomposition for V
    eigvals_u, eigvecs_u = calculate_eigen(aat)  # Eigen decomposition for U

    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices_v = np.argsort(eigvals_v)[::-1]
    sorted_indices_u = np.argsort(eigvals_u)[::-1]

    eigvals_v = eigvals_v[sorted_indices_v]
    eigvecs_v = eigvecs_v[:, sorted_indices_v]
    eigvals_u = eigvals_u[sorted_indices_u]
    eigvecs_u = eigvecs_u[:, sorted_indices_u]

    # Create Σ with square roots of eigenvalues
    singular_values = np.sqrt(np.abs(eigvals_v))
    sigma = np.zeros_like(matrix, dtype=float)
    np.fill_diagonal(sigma, singular_values)

    u = eigvecs_u
    vt = eigvecs_v.T
    return u, sigma, vt

def verify_svd(matrix, u, sigma, vt):
    """
    Verify if A = U Σ V^T using simple equality check.
    """
    reconstructed = np.dot(np.dot(u, sigma), vt)
    print(reconstructed)
    return np.array_equal(matrix, reconstructed)

def main():
    """
    Main function to calculate eigenvalues, eigenvectors, and SVD.
    """
    print("Enter the matrix:")
    num_rows, num_cols = row_column_input()
    A = input_matrix(num_rows, num_cols)

    if not is_svd_possible(A):
        return

    # SVD
    try:
        U, Sigma, VT = svd_decomposition(A)
        print("U:\n", U)
        print("Sigma:\n", Sigma)
        print("VT:\n", VT)

        # Verification
        if verify_svd(A, U, Sigma, VT):
            print("SVD verified successfully: A = U Σ V^T")
        else:
            print("SVD verification failed.")
    except Exception as e:
        print("SVD could not be performed:", str(e))

if __name__ == "__main__":
    main()
|

Enter the matrix:
U:
 [[-0.40455358 -0.9145143 ]
 [-0.9145143   0.40455358]]
Sigma:
 [[5.4649857  0.        ]
 [0.         0.36596619]]
VT:
 [[-0.57604844 -0.81741556]
 [-0.81741556  0.57604844]]
[[1.54714743 1.61441471]
 [2.75795846 4.17057132]]
SVD verification failed.


In [7]:
import numpy as np

# Define the input matrix
A = np.array([[1, 2],
              [3, 4]])

# Perform SVD
U, S, VT = np.linalg.svd(A)

# Print the results
print("U:")
print(U)

print("\nS:")
print(S)

print("\nVT:")
print(VT)

# Reconstruct the original matrix using SVD
A_reconstructed = np.dot(U, np.dot(np.diag(S), VT))
print("\nReconstructed matrix:")
print(A_reconstructed)

U:
[[-0.40455358 -0.9145143 ]
 [-0.9145143   0.40455358]]

S:
[5.4649857  0.36596619]

VT:
[[-0.57604844 -0.81741556]
 [ 0.81741556 -0.57604844]]

Reconstructed matrix:
[[1. 2.]
 [3. 4.]]


In [ ]:
import numpy as np

def row_column_input():
    check = True
    while check:
        try:
            num_of_row = int(input("Enter number of rows: "))
            num_of_col = int(input("Enter number of columns: "))
            check = False
        except ValueError as e:
            print(f"Error: {e}.")
    return num_of_row, num_of_col

def input_matrix(num_rows, num_cols):
    matrix = []
    i = 0
    while i < num_rows:
        try:
            row = list(map(int, input(f"Row {i+1}, enter values separated by space ({num_cols}): ").split()))
            if num_cols != len(row):
                raise ValueError(f"Expected {num_cols} values but got {len(row)} values.")
            matrix.append(row)
            i += 1  
        except ValueError as e:
            print(f"Error: {e}. Please re-enter Row {i+1}.")
    return np.array(matrix)

def is_svd_possible(matrix):
    """
    Check if SVD is possible: non-empty, numerical, and 2D.
    """
    if matrix.size == 0:
        print("Matrix is empty!")
        return False
    if len(matrix.shape) != 2:
        print("Matrix is not 2D!")
        return False
    return True

def calculate_eigen(matrix):
    """
    Calculate eigenvalues and eigenvectors of a given matrix.
    """
    eigvals, eigvecs = np.linalg.eig(matrix)
    return eigvals, eigvecs

def svd_decomposition(matrix):
    """
    Perform SVD decomposition by manually calculating U, Σ, and V^T.
    """
    ata = np.dot(matrix.T, matrix)  # A^T * A
    aat = np.dot(matrix, matrix.T)  # A * A^T

    eigvals_v, eigvecs_v = calculate_eigen(ata)  # Eigen decomposition for V^T
    eigvals_u, eigvecs_u = calculate_eigen(aat)  # Eigen decomposition for U

    # Sort eigenvalues and eigenvectors in descending order
    sorted_indices_v = np.argsort(eigvals_v)[::-1]
    sorted_indices_u = np.argsort(eigvals_u)[::-1]

    eigvals_v = eigvals_v[sorted_indices_v]
    eigvecs_v = eigvecs_v[:, sorted_indices_v]
    eigvals_u = eigvals_u[sorted_indices_u]
    eigvecs_u = eigvecs_u[:, sorted_indices_u]

    # Create Σ with square roots of eigenvalues (singular values)
    singular_values = np.sqrt(np.abs(eigvals_v))  # Singular values are sqrt of eigenvalues
    sigma = np.zeros_like(matrix, dtype=float)
    np.fill_diagonal(sigma, singular_values)

    u = eigvecs_u
    vt = eigvecs_v.T
    return u, sigma, vt

def verify_svd(matrix, u, sigma, vt):
    """
    Verify if A = U Σ V^T using simple equality check.
    """
    reconstructed = np.dot(np.dot(u, sigma), vt)
    return np.array_equal(matrix, reconstructed)

def main():
    num_rows, num_cols = row_column_input()
    A = input_matrix(num_rows, num_cols)

    if not is_svd_possible(A):
        return

    # SVD
    try:
        U, Sigma, VT = svd_decomposition(A)
        print("U:\n", U)
        print("Sigma:\n", Sigma)
        print("VT:\n", VT)

        # Verification
        if verify_svd(A, U, Sigma, VT):
            print("SVD verified successfully: A = U Σ V^T")
        else:
            print("SVD verification failed.")
    except Exception as e:
        print("SVD could not be performed:", str(e))

if __name__ == "__main__":
    main()


Enter the matrix:
U:
 [[-0.40455358 -0.9145143 ]
 [-0.9145143   0.40455358]]
Sigma:
 [[5.4649857  0.        ]
 [0.         0.36596619]]
VT:
 [[-0.57604844 -0.81741556]
 [-0.81741556  0.57604844]]
SVD verification failed.


In [ ]:
import numpy as np

def row_column_input():
    check = True
    while check:
        try:
            num_of_row = int(input("Enter number of rows: "))
            num_of_col = int(input("Enter number of columns: "))
            check = False
        except ValueError as e:
            print(f"Error: {e}.")
    return num_of_row, num_of_col

def input_matrix(num_rows, num_cols):
    matrix = []
    i = 0
    while i < num_rows:
        try:
            row = list(map(int, input(f"Row {i+1}, enter values separated by space ({num_cols}): ").split()))
            if num_cols != len(row):
                raise ValueError(f"Expected {num_cols} values but got {len(row)} values.")
            matrix.append(row)
            i += 1  
        except ValueError as e:
            print(f"Error: {e}. Please re-enter Row {i+1}.")
    return np.array(matrix)

def is_svd_possible(matrix):
    if matrix.size == 0:
        print("Matrix is empty!")
        return False
    if len(matrix.shape) != 2:
        print("Matrix is not 2D!")
        return False
    return True

def calculate_eigenvalues_and_vectors(matrix):
    eigenvalues, eigenvectors = np.linalg.eig(matrix)
    return eigenvalues, eigenvectors

def svd_decomposition(A):
    AtA = np.dot(A.T, A)
    eigenvalues_AtA, V = calculate_eigenvalues_and_vectors(AtA)
    singular_values = np.sqrt(np.abs(eigenvalues_AtA))
    sorted_indices = np.argsort(singular_values)[::-1]
    singular_values = singular_values[sorted_indices]
    V = V[:, sorted_indices]
    m, n = A.shape
    Sigma = np.zeros((m, n))
    np.fill_diagonal(Sigma, singular_values)
    U = np.zeros((m, m))
    for i in range(len(singular_values)):
        if singular_values[i] > 1e-10:
            U[:, i] = (1 / singular_values[i]) * np.dot(A, V[:, i])
    return U, Sigma, V

def verify_svd(A, U, Sigma, V):
    A_reconstructed = np.dot(U, np.dot(Sigma, V.T))
    return np.allclose(A, A_reconstructed)

def main():
    print("Enter the matrix:")
    num_rows, num_cols = row_column_input()
    A = input_matrix(num_rows, num_cols)

    if not is_svd_possible(A):
        return

    try:
        U, Sigma, V = svd_decomposition(A)
        print("U:\n", U)
        print("Sigma:\n", Sigma)
        print("V^T:\n", V.T)

        if verify_svd(A, U, Sigma, V):
            print("SVD verified successfully: A = U Σ V^T")
        else:
            print("SVD verification failed.")
    except Exception as e:
        print("SVD could not be performed:", str(e))

if __name__ == "__main__":
    main()


Enter the matrix:
U:
 [[-0.40455358  0.9145143 ]
 [-0.9145143  -0.40455358]]
Sigma:
 [[5.4649857  0.        ]
 [0.         0.36596619]]
V^T:
 [[-0.57604844 -0.81741556]
 [-0.81741556  0.57604844]]
[[1. 2.]
 [3. 4.]]
SVD verified successfully: A = U Σ V^T
